Bert는 사전 학습된 대용량의 레이블링이 되지 않은 데이터를 이용하여 언어모델을 학습, 이를 토대로 문서분류, 질의응답, 번역 등의 작업을 위한 신경망을 추가하는 전이 학습 방법

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.utils import resample

KeyboardInterrupt: 

In [ ]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
submit = pd.read_csv('./sample_submission.csv')

train['contents'] = train['first_party'] + ',' + train['second_party'] + ',' + train['facts']
test['contents'] = test['first_party'] + ',' + test['second_party'] + ','+ test['facts']

train = train[['contents','first_party_winner']]
test = test[['contents']]

In [ ]:
sns.distplot(train['first_party_winner'], kde=True)

In [ ]:
# 0과 1의 데이터가 불귢녕 --> 1 다운 샘플링
subset_0 = train[train['first_party_winner']==0]
subset_1 = train[train['first_party_winner']==1]

subset_1_downsampled = resample(subset_1, replace=False, n_samples=800, random_state=42)
train = pd.concat([subset_0, subset_1_downsampled])

In [ ]:
sns.distplot(train['first_party_winner'], kde=True)

In [ ]:
train.head()

In [ ]:
y = train['first_party_winner']
x = train['contents']

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x,y,stratify=y, test_size=0.3, random_state=42)

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
# Initializing bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

In [ ]:
#Initializing neural network layers
I = tf.keras.layers.Dropout(0.1, name='dropout')(outputs['pooled_output'])
I = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(I)

In [ ]:
# Create model
model = tf.keras.Model(inputs=[text_input], outputs=[I])
model.summary()

In [ ]:
METRICS = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy')
]

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=METRICS)

In [ ]:
model.fit(x_train, y_train, epochs=10)

In [ ]:
y_pred = model.predict(test)

submit["first_party_winner"] = y_pred
submit["first_party_winner"] = submit["first_party_winner"].apply(lambda x: 1 if x >= 0.5 else 0)
submit.to_csv("bert_submit.csv", index=False)